In [17]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import numpy as np
import matplotlib.image as mpimg
import cv2
from keras.utils.vis_utils import plot_model

In [18]:
 model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [13]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 4, 4, 64)         

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=30, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=True,rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory('F:\\class dataset2\\train',target_size=(64, 64), batch_size=100,class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory('F:\\class dataset2\\test',target_size=(64, 64), batch_size=100,class_mode='categorical')


In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam,SGD

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
path_train = 'F:\\class dataset2\\train'
path_test = 'F:\\class dataset2\\test'
label_train=[]
label_test=[]
for filename in os.listdir(path_train):
        label_train.append(filename)
for filename in os.listdir(path_test):
        label_test.append(filename)
print(label_train)
print(label_test)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.9) and (logs.get('accuracy')>0.92):
            print("\nReached 90% val_accuracy so cancelling training!")
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
checkpoint = ModelCheckpoint('Classifier2_ADAM.h5',monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=36,  
      epochs=30,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=4,
      callbacks=[callbacks,callback])

In [ ]:
from keras.preprocessing import image
path = 'F:\\class dataset2\\test\\uma\\932.jpg'
img = image.load_img(path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
model1 = tf.keras.models.load_model('F:\\Final_Classifier1.h5')
classes = model1.predict(images)
print(label_train[np.argmax(classes)])
output = plt.imread(path)
plt.imshow(output)
plt.show()
    

In [ ]:
results = model.evaluate(validation_generator, batch_size=10)
results1 = model.evaluate(train_generator, batch_size=10)
print("test loss, test acc:", results)
print("train loss, train acc:", results1)

In [ ]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('Final_Classifier1.h5')

In [ ]:
model.save("Final_Classifier1.h5")

In [ ]:
from ann_visualizer.visualize import ann_viz;
from keras.models import model_from_json
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
ann_viz(model, title="Artificial Neural network - Model Visualization")